In [1]:
import duckdb
import yaml
import datetime
import openai

In [2]:
with open("config.yaml", "r") as stream:
    try:
        PARAM = yaml.safe_load(stream)
    except yaml.YAMLError as exc:
        print(exc)

openai.api_key  = PARAM['openai_api']
client = openai.OpenAI(api_key = PARAM['openai_api'])

def get_embedding(text, model=PARAM['vector_embedding_model']):
   text = text.replace("\n", " ")
   return client.embeddings.create(input = [text], model=model).data[0].embedding

In [3]:
con = duckdb.connect()
con.execute("install duckpgq from community;")
con.execute("load duckpgq;")

con.execute("install ducklake;")
con.execute("load ducklake;")

con.execute("install httpfs;")
con.execute("load httpfs;")

con.execute("install postgres;")
con.execute("load postgres;")

In [4]:
con.sql(f"""

CREATE SECRET(

TYPE postgres,

HOST '{PARAM["supabase_postgres_host"]}',

PORT 5432,

DATABASE postgres,

USER '{PARAM["supabase_postgres_user"]}',

PASSWORD '{PARAM["supabase_postgres_password"]}'

);

""")

┌─────────┐
│ Success │
│ boolean │
├─────────┤
│ true    │
└─────────┘

In [5]:
con.sql(f"""

CREATE OR REPLACE SECRET secret (

TYPE gcs,

KEY_ID '{PARAM["gcp_access_key"]}',

SECRET '{PARAM["gcp_secret"]}',

REGION 'asia-northeast1'

);

""")



┌─────────┐
│ Success │
│ boolean │
├─────────┤
│ true    │
└─────────┘

In [6]:

con.sql(f"""
  ATTACH 'ducklake:postgres:dbname=postgres' AS sixing_ducklake (
      DATA_PATH 'gcs://{PARAM["gcp_bucket_name"]}/ducklake/',
      SNAPSHOT_VERSION 5
  );
""")

con.sql("""
USE sixing_ducklake;
        """)

In [7]:
con.sql(f"""
    SHOW TABLES;
""")

┌──────────────┐
│     name     │
│   varchar    │
├──────────────┤
│ Disease      │
│ Drug         │
│ Drug_Disease │
│ Drug_MOA     │
│ MOA          │
└──────────────┘

In [8]:
con.sql(f"""
    SELECT *
    FROM Trial
    LIMIT 5;
""")

CatalogException: Catalog Error: Table with name Trial does not exist!
Did you mean "Disease"?

In [ ]:
con.close()